In [1]:
import iyer_saliency
import sarfa_saliency

import chess 
import chess.uci
import time

from sys import platform as _platform
from collections import defaultdict


from IPython.display import Image, display
import cairosvg
import cv2
import svg_custom.svg_custom as svg_custom 
import matplotlib.pyplot as plt
import numpy as np

import json 
import pandas as pd 
from pandas.io.json import json_normalize 



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The chess.uci module is deprecated in favor of
chess.engine <https://python-chess.readthedocs.io/en/latest/engine.html>.

Please consider updating and open an issue
<https://github.com/niklasf/python-chess/issues/new> if your use case
is not covered by the new API.
  """


In [2]:
def select_engine():
    handler = chess.uci.InfoHandler()
    if _platform == "linux" or _platform == "linux2":

        engine = chess.uci.popen_engine('engines/stockfish-11-linux/stockfish-11-linux/Linux/stockfish_20011801_x64')

    elif _platform == "darwin":
        engine = chess.uci.popen_engine('engines/stockfish-11-mac/stockfish-11-mac/Mac/stockfish-11-64')

    elif _platform == "win32":
        engine = chess.uci.popen_engine('engines/stockfish-11-win/stockfish-11-win/Windows/stockfish_20011801_32bit.exe') 

    elif _platform == "win64":
        engine = chess.uci.popen_engine('engines/stockfish-11-win/stockfish-11-win/Windows/stockfish_20011801_x64.exe')
    engine.setoption({'MultiPV': 100})
    engine.info_handlers.append(handler)
    return handler,engine

In [3]:

def get_dict_q_vals(board, legal_moves, eval_time, original_move,engine,handler):
    """
    Function returns Q-values in given board position
    Input:
        board: chess.Board()
        legal_moves: List of legal moves of original state
        eval_time: Search time for stockfish
        original_move: original best move (chess.Move()) 
    Output:
        q_vals_dict: Dictionary containing Q-values and Actions
        bestmove: chess.Move() - Best move in given board position
    """
    
    i = 0
    q_vals_dict = {}
    
    
    set_current_legal_moves = set(board.legal_moves)
    set_original_legal_moves = set(legal_moves)
    intersection_set = set_current_legal_moves.intersection(set_original_legal_moves)

    #print('querying engine with perturbed position')
    engine.position(board)
    evaluation = engine.go(movetime=eval_time)
    #print(evaluation)
    if original_move is None:
        # no initial move supplied
        original_move = evaluation.bestmove
    dict_moves_to_score = defaultdict(int)
    
    for move_id in handler.info['pv'].keys():
        move_string = str(handler.info['pv'][move_id][0])
        move_score = 0
        if handler.info["score"][move_id].cp is None:
            mate_in_moves = handler.info["score"][move_id].mate
            if mate_in_moves > 0:
                # white will win in some number of moves
                move_score = 40
            else:
                # black will win 
                move_score = -40
        else:
            move_score = round(handler.info["score"][move_id].cp/100.0,2)
        dict_moves_to_score[move_string] = move_score
    
    #print(dict_moves_to_score)
    
    #print('Total Legal Moves : ', len(intersection_set))

    for el in legal_moves:
        if el in intersection_set:
            i += 1
            score = dict_moves_to_score[str(el)]
            q_vals_dict[el.uci()] = score
    
    return q_vals_dict, evaluation.bestmove

In [4]:
def computeSaliency(FEN = 'rnbq1rk1/pp2bppp/4p3/3p3n/3P1B2/3B1N2/PPPNQPPP/R3K2R w KQkq - 0 1',typeSal='sarfa',engine= chess.uci.popen_engine('engines/stockfish-11-linux/stockfish-11-linux/Linux/stockfish_20011801_x64'),handler=None):
    """
    Function returns saliency map for given board position  
    Input:
        FEN : Board position encoded in a FEN  
    Output:
        answer : Saliency for each location on the board
    """

    print("***********************", FEN, "**********************")
    board = chess.Board(FEN)
    evaltime = 6000
    legal_moves = list(board.legal_moves)[:]

    # Q-values for original state
    dict_q_values_before_perturbation, original_move  = get_dict_q_vals(board, legal_moves, evaltime, None,engine,handler)
    #print('original move = ', original_move)
    
    # Saliency for each board location
    answer = {
        'a1' : {'int': chess.A1, 'saliency': -2},
        'a2' : {'int': chess.A2, 'saliency': -2},
        'a3' : {'int': chess.A3, 'saliency': -2},
        'a4' : {'int': chess.A4, 'saliency': -2},
        'a5' : {'int': chess.A5, 'saliency': -2},
        'a6' : {'int': chess.A6, 'saliency': -2},
        'a7' : {'int': chess.A7, 'saliency': -2},
        'a8' : {'int': chess.A8, 'saliency': -2},
        'b1' : {'int': chess.B1, 'saliency': -2},
        'b2' : {'int': chess.B2, 'saliency': -2},
        'b3' : {'int': chess.B3, 'saliency': -2},
        'b4' : {'int': chess.B4, 'saliency': -2},
        'b5' : {'int': chess.B5, 'saliency': -2},
        'b6' : {'int': chess.B6, 'saliency': -2},
        'b7' : {'int': chess.B7, 'saliency': -2},
        'b8' : {'int': chess.B8, 'saliency': -2},
        'c1' : {'int': chess.C1, 'saliency': -2},
        'c2' : {'int': chess.C2, 'saliency': -2},
        'c3' : {'int': chess.C3, 'saliency': -2},
        'c4' : {'int': chess.C4, 'saliency': -2},
        'c5' : {'int': chess.C5, 'saliency': -2},
        'c6' : {'int': chess.C6, 'saliency': -2},
        'c7' : {'int': chess.C7, 'saliency': -2},
        'c8' : {'int': chess.C8, 'saliency': -2},
        'd1' : {'int': chess.D1, 'saliency': -2},
        'd2' : {'int': chess.D2, 'saliency': -2},
        'd3' : {'int': chess.D3, 'saliency': -2},
        'd4' : {'int': chess.D4, 'saliency': -2},
        'd5' : {'int': chess.D5, 'saliency': -2},
        'd6' : {'int': chess.D6, 'saliency': -2},
        'd7' : {'int': chess.D7, 'saliency': -2},
        'd8' : {'int': chess.D8, 'saliency': -2},
        'e1' : {'int': chess.E1, 'saliency': -2},
        'e2' : {'int': chess.E2, 'saliency': -2},
        'e3' : {'int': chess.E3, 'saliency': -2},
        'e4' : {'int': chess.E4, 'saliency': -2},
        'e5' : {'int': chess.E5, 'saliency': -2},
        'e6' : {'int': chess.E6, 'saliency': -2},
        'e7' : {'int': chess.E7, 'saliency': -2},
        'e8' : {'int': chess.E8, 'saliency': -2},
        'f1' : {'int': chess.F1, 'saliency': -2},
        'f2' : {'int': chess.F2, 'saliency': -2},
        'f3' : {'int': chess.F3, 'saliency': -2},
        'f4' : {'int': chess.F4, 'saliency': -2},
        'f5' : {'int': chess.F5, 'saliency': -2},
        'f6' : {'int': chess.F6, 'saliency': -2},
        'f7' : {'int': chess.F7, 'saliency': -2},
        'f8' : {'int': chess.F8, 'saliency': -2},
        'g1' : {'int': chess.G1, 'saliency': -2},
        'g2' : {'int': chess.G2, 'saliency': -2},
        'g3' : {'int': chess.G3, 'saliency': -2},
        'g4' : {'int': chess.G4, 'saliency': -2},
        'g5' : {'int': chess.G5, 'saliency': -2},
        'g6' : {'int': chess.G6, 'saliency': -2},
        'g7' : {'int': chess.G7, 'saliency': -2},
        'g8' : {'int': chess.G8, 'saliency': -2},
        'h1' : {'int': chess.H1, 'saliency': -2},
        'h2' : {'int': chess.H2, 'saliency': -2},
        'h3' : {'int': chess.H3, 'saliency': -2},
        'h4' : {'int': chess.H4, 'saliency': -2},
        'h5' : {'int': chess.H5, 'saliency': -2},
        'h6' : {'int': chess.H6, 'saliency': -2},
        'h7' : {'int': chess.H7, 'saliency': -2},
        'h8' : {'int': chess.H8, 'saliency': -2},

        }
    
    # Iteratively perturb each feature on the board
    # Note : Perturbations should be valid. Code for avoiding those cases.
    
    for square_string in sorted(answer.keys()):
        entry = answer[square_string]
        entry_keys = ['saliency']
        #print('perturbing square = ', square_string)
        # perturb board
        piece_removed = board.remove_piece_at(entry['int'])
        
        if piece_removed is None:
            # square was empty, so proceed without changing anything
            #print('square was empty, so skipped')
            # print(board)
            #print('------------------------------------------')
        
            continue
        
        elif (piece_removed == chess.Piece(6,True) or piece_removed == chess.Piece(6,False)) or board.was_into_check():
            # illegal piece was removed
            #print('illegal piece was removed')
            for key in entry_keys:
                entry[key] = 0
        else:
            # set perturbed state
            engine.position(board)

            # Check if the original move is still valid
            if board.is_legal(original_move):
                # Find the q values 
                dict_q_values_after_perturbation, _ = get_dict_q_vals(board, legal_moves, evaltime, original_move,engine,handler)
                if typeSal=='sarfa':
                    '''entry['saliency'], entry['dP'], entry['K'], entry['QMaxAnswer'],\
                    entry['actionGapBeforePerturbation'], entry['actionGapAfterPerturbation']\
                         = sarfa_saliency.computeSaliencyUsingSarfa(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)'''
                    entry['saliency'], entry['dP'], entry['K'] =sarfa_saliency.computeSaliencyUsingSarfa(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)
              
                else:
                    entry['saliency']=iyer_saliency.computeSaliencyUsingIyer(str(original_move), dict_q_values_before_perturbation, dict_q_values_after_perturbation)

            else:
                # illegal original move in perturbed state, therefore piece removed is probably important 
                # print(board.is_legal(original_move))
                # print(board)
                #print('original move illegal in perturbed state')
                for key in entry_keys:
                    entry[key] = -1
                entry['saliency'] = 1 
                
        # undo perturbation
        #print('------------------------------------------')
                
        board.set_piece_at(entry['int'], piece_removed)
        

    #print('BBBOB',answer)
    return(answer)

In [5]:
# Cell for displaying board position

# Few utility functions
def svg_to_png(img):
    '''
    Converts given svg image to png
    Input : 
        img : image in .svg format 
    Output :
        svg_custom/board.png
        Display of image
    '''
    with open('svg_custom/board.svg', 'w+') as f:
        f.write(img)
    cairosvg.svg2png(url='svg_custom/board.svg', write_to='svg_custom/board.png')
    display(Image(filename='svg_custom/board.png'))

def display_board(board):
    '''
    Displaying given board
    Input : 
        board : chess.Board
    Output :
        svg_custom/board.png
        Display of image     
    '''
    img = svg_custom.board(board)
    svg_to_png(img)
    
def return_bestmove(board, eval_time = 6000):
    '''
    Returns and displays best move for a given chess position
    Input :
        board : chess.Board
    Output :
        bestmove : chess.Move 
    '''
    engine.position(board)
    bestmove =  engine.go(movetime=eval_time).bestmove
    #print('Best move is', bestmove)
    
    svg_w_arrow = svg_custom.board(board, arrows = [svg_custom.Arrow(tail =  bestmove.from_square, head = bestmove.to_square, color = '#e6e600')])
    
    svg_to_png(svg_w_arrow)
    
    return bestmove

def explanation(board,typeS,engine,handler):
    '''
    Generates explanation of the best move for given board position using SARFA
    
    Input :
        board = chess.Board
    Output :
        prints piecewise saliency
        svg_custom/board.png
    '''
    
    bestmove =  engine.go(movetime=6000).bestmove

    # Evaluation of board position for best move
    evaluation = computeSaliency(chess.Board.fen(board),typeS,engine,handler)
    #print('EVALUATIIIION',evaluation)

    return evaluation, bestmove 

def generate_heatmap(evaluation, bestmove):
    """
    Generates heatmap for  saliency evaluation of the best move

    """
    # Laying the saliency map over the board
    heatmap = np.zeros((8, 8))
    for position in evaluation:
        x, y = evaluation[position]['int']//8, evaluation[position]['int'] % 8
        heatmap[x, y] = evaluation[position]['saliency']
    heatmap = np.flipud(heatmap)
    #è una matrice 8X8 dove all'interno ci sono i valori delle saliency per ogni casella    

    #### Saliency map overlaid on board
    svg = svg_custom.board(board, arrows = [svg_custom.Arrow(tail =  bestmove.from_square, head = bestmove.to_square, color = '#e6e600')])

    with open('svg_custom/board.svg', 'w+') as f:
        f.write(svg)
    cairosvg.svg2png(url='svg_custom/board.svg', write_to='svg_custom/board.png')

    # original board as a numpy array
    board_array = cv2.imread('svg_custom/board.png')

    threshold = (100/256)*np.max(heatmap) # percentage threshold. Saliency values above this threshold won't be mapped onto board

    #miserve(evaluation,threshold)
    
    
    # Create bounding boxes with saliency colours for every square on chess board
    for i in range(0, 8, 1):
        for j in range(0, 8, 1):
            ii = 45*i+20
            jj = 45*j+20
            value_of_square =  heatmap[i, j]
            if value_of_square < threshold:
                continue
            for box_i in range(ii, ii+44, 1):
                for box_j in range(jj, jj+44, 1):
                    if box_i > ii+4 and box_i < ii+40 and box_j > jj+4 and box_j < jj+40:
                        continue
                    board_array[box_i, box_j, 0] = 256 - 0.8*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)
                    board_array[box_i, box_j, 1] = 256 - 0.84*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)
                    board_array[box_i, box_j, 2] = 256 - 0.19*256*heatmap[i, j]/(np.max(heatmap) + 1e-10)

    cv2.imwrite("svg_custom/board.png", board_array)
    #print('------------===============------------===============\n\n')
    display(Image('svg_custom/board.png'))
    #print('\n\n------------===============------------===============')

In [6]:
def saliency(evaluation,bestmove):
    # Laying the saliency map over the board
    heatmap = np.zeros((8, 8))
    for position in evaluation:
        x, y = evaluation[position]['int']//8, evaluation[position]['int'] % 8
        heatmap[x, y] = evaluation[position]['saliency']
    heatmap = np.flipud(heatmap)
    #è una matrice 8X8 dove all'interno ci sono i valori delle saliency per ogni casella    
    
    threshold = (100/256)*np.max(heatmap) # percentage threshold. Saliency values above this threshold won't be mapped onto board

    dict_sarfa={}
    for k in evaluation.keys():
        if evaluation[k]['saliency'] >= threshold:
            dict_sarfa[k]=evaluation[k]['saliency']
    return dict_sarfa

In [7]:
def gameOne(args):
    global counterS,counterI
    FEN,t = args

    handler,engine=select_engine()
    board = chess.Board(FEN)
    evaluation, bestmove = explanation(board,t,engine,handler)
    dict_sal= saliency(evaluation,bestmove)
    max_key = max(dict_sal, key=dict_sal.get)
    df_explode=df_sarfa.explode('saliencyGroundTruth')
    if df_explode.loc[(df_explode['fen']==FEN)]['saliencyGroundTruth'].str.contains(max_key).any():
        #print(t,' Saliency is into Dataset of expert')
        if t=='sarfa':
            with counterS.get_lock():
                    counterS.value += 1
        else:
            with counterI.get_lock():
                    counterI.value += 1
    return 1


def gameMulti(args):
    global counterSMulti,counterIMulti
    FEN,t = args

    handler,engine=select_engine()
    board = chess.Board(FEN)
    evaluation, bestmove = explanation(board,t,engine,handler)
    dict_sal= saliency(evaluation,bestmove)
    df_explode=df_sarfa.explode('saliencyGroundTruth')
    cc=df_explode.loc[(df_explode['fen']==FEN)]['saliencyGroundTruth'].values
    difference=set(dict_sal)^set(cc)
    #qui molto easy, se hai mancato o messo qualcosa in più ti tolgo un punto
    score=(len(cc)-len(difference))/len(cc)
    if t=='sarfa':
        with counterSMulti.get_lock():
            #print("pippo",score)
            counterSMulti = counterSMulti.value + score
    else:
        with counterIMulti.get_lock():
            counterIMulti = counterIMulti.value + score
    return 1


from multiprocessing import Pool, Value
from time import sleep

counterS = None
counterI = None

counterSMulti = 0
counterIMulti = 0

if __name__ == '__main__':
    #inputs = os.listdir(some_directory)

    #
    # initialize a cross-process counter and the input lists
    #
    #load json object
    with open('chess_saliency_databases/chess-saliency-dataset-v1.json') as f:
        d = json.load(f)

    df_sarfa = pd.json_normalize(d['puzzles'])
    args=[]
    number=2
    for t in ['sarfa','iyer']:
        for i in range(0,number):
            args.append((df_sarfa['fen'][i],t))
    counterS = Value('i', 0)
    counterI = Value('i', 0)
    counterSMulti = Value('f', 0)
    counterIMulti = Value('f', 0)

      
    with Pool(64) as p:
        i=p.map_async(gameOne, args,chunksize=1)
        i.wait()
    '''    
    with Pool(64) as p:
        i=p.map_async(gameMulti, args,chunksize=1)
        i.wait()
    '''


        


print('Percentuale di accuracy per Sarfa:',counterS.value/number)
print('Percentuale di accuracy per Iyer:',counterI.value/number)

In [8]:
with open('chess_saliency_databases/chess-saliency-dataset-v1.json') as f:
    d = json.load(f)

df_sarfa = pd.json_normalize(d['puzzles'])

In [9]:
number=1
for t in ['sarfa']:#,'iyer']:
    counters=0
    counteri=0
    for index, row in df_sarfa.head(number).iterrows():
        FEN = row['fen']
        board = chess.Board(FEN)
        handler,engine=select_engine()
        evaluation, bestmove = explanation(board,t,engine,handler)
        dict_sal= saliency(evaluation,bestmove)
        max_key = max(dict_sal, key=dict_sal.get)
        df_explode=df_sarfa.explode('saliencyGroundTruth')
        #cc=df_explode.loc[(df_explode['fen']==FEN)]['saliencyGroundTruth'].values
        #difference=set(dict_sal)^set(cc)
        #qui molto easy, se hai mancato o messo qualcosa in più ti tolgo un punto
        #score=(len(cc)-len(difference))/len(cc)
        #print(score)
        file_name=f'file/{number}.txt'
        #import os
        #if not (os.path.isfile(file_name)):
        if t=='sarfa':
            with open(file_name,"w") as f:
                f.write(f'FEN:{FEN} \n')
                etc=df_sarfa.loc[(df_sarfa['fen']==FEN)]['saliencyGroundTruth'].tolist()
                f.write(f'Most salient cells for experts:{etc}\n')
                f.write(f'Most salient cells for {t}: \n')
                for key, value in dict_sal.items(): 
                    f.write('%s:%s\n' % (key, value))
                f.write(f'Information about P and K: \n')
                for key, value in evaluation.items(): 
                    f.write('%s:%s\n' % (key, value))
                
                    
        else:
            with open(file_name,"a") as f:
                f.write(f'Most salient cells for {t}: \n')
                for key, value in dict_sal.items(): 
                    f.write('%s:%s\n' % (key, value))
                
            
        

           



*********************** 2r2rk1/pp1bqpp1/2nppn1p/2p3N1/1bP5/1PN3P1/PBQPPPBP/3R1RK1 w - - 0 1 **********************
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action for KL-Divergence
skipping original action 

In [10]:
df_sarfa.loc[(df_sarfa['fen']==FEN)]['saliencyGroundTruth'].tolist()

[['c3', 'c2', 'f8', 'b2', 'f7', 'e7', 'g5']]

In [11]:
    evaluation['a1']['saliency']

-2

In [12]:
FEN

'2r2rk1/pp1bqpp1/2nppn1p/2p3N1/1bP5/1PN3P1/PBQPPPBP/3R1RK1 w - - 0 1'

In [13]:
for key, value in evaluation.items(): 
    print(key,value)


a1 {'int': 0, 'saliency': -2}
a2 {'int': 8, 'saliency': 0.024294044097056912, 'dP': 0.012303523771582503, 'K': 0.9549490677262052}
a3 {'int': 16, 'saliency': -2}
a4 {'int': 24, 'saliency': -2}
a5 {'int': 32, 'saliency': -2}
a6 {'int': 40, 'saliency': -2}
a7 {'int': 48, 'saliency': 0.042527274854980895, 'dP': 0.021730562627759853, 'K': 0.9896034836960073}
a8 {'int': 56, 'saliency': -2}
b1 {'int': 1, 'saliency': -2}
b2 {'int': 9, 'saliency': 0.8017004226405682, 'dP': 0.9546554624560035, 'K': 0.6909899158210975}
b3 {'int': 17, 'saliency': 0, 'dP': -0.0036085402724322924, 'K': 0.9349374879215515}
b4 {'int': 25, 'saliency': 0.10890209800043942, 'dP': 0.057762915402906234, 'K': 0.9496914895020581}
b5 {'int': 33, 'saliency': -2}
b6 {'int': 41, 'saliency': -2}
b7 {'int': 49, 'saliency': 0, 'dP': -0.005025912571889668, 'K': 0.8795993129972698}
b8 {'int': 57, 'saliency': -2}
c1 {'int': 2, 'saliency': -2}
c2 {'int': 10, 'saliency': 0.8466727206567926, 'dP': 0.9565968796613686, 'K': 0.759407881618